# Visualización de Faros de Galicia
Este notebook crea un mapa interactivo de los faros de Galicia utilizando datos de OpenStreetMap.

In [1]:
import geopandas
import overpass

Realizamos una consulta a la API de Overpass para obtener los faros de Galicia:
- Definimos el área de búsqueda (Galicia)
- Buscamos nodos con la etiqueta "man_made=lighthouse"
- Solicitamos los datos en formato GeoJSON

In [2]:
api = overpass.API()
faros = api.get("""
area[name = "Galicia"];
node["man_made" = "lighthouse"](area);
out body;""", responseformat="geojson")

Convertimos los datos GeoJSON a un GeoDataFrame de geopandas para facilitar su manipulación.

In [3]:
farosData= geopandas.GeoDataFrame.from_features(faros.features)

Creamos una primera visualización simple:
- Puntos amarillos representando los faros
- Fondo negro para mejor contraste
- Tamaño de figura grande (60x60)

In [ ]:
ax = farosData.plot(color="yellow", figsize=(60, 60))
ax.set_facecolor('black')

Creamos un mapa interactivo más elaborado usando folium:

### Características del mapa:
1. **Mapa Base**: 
   - Centrado en Galicia [42.8, -8.5]
   - Zoom inicial nivel 8
   - Estilo CartoDB positron por defecto

2. **Marcadores**:
   - Icono de faro en color naranja
   - Popup con información detallada:
     - Nombre del faro
     - Características técnicas
     - Alcance y altura
     - Período y secuencia de luz

3. **Elementos adicionales**:
   - Mini mapa para orientación
   - Control de escala y medición
   - Diferentes capas base disponibles
   - Control de capas

Exportamos el código del visor a un fichero html, que podemos ejecutar directamente desde un navegador web.

In [ ]:
import folium
from folium import plugins

farosData.crs = "EPSG:4326"

# Creamos el mapa centrado en Galicia
m = folium.Map(location=[42.8, -8.5], 
               zoom_start=8,
               tiles='CartoDB positron')

# Añadimos los faros al mapa
for idx, row in farosData.iterrows():
    # Preparamos el popup con la información disponible
    popup_text = "<b>Faro</b><br>"
    if 'name' in row:
        popup_text += f"Nombre: {row['name']}<br>"
    if 'alt_name' in row:
        popup_text += f"Nombre alternativo: {row['alt_name']}<br>"
    if 'seamark:light:character' in row:
        popup_text += f"Característica: {row['seamark:light:character']}<br>"
    if 'seamark:light:colour' in row:
        popup_text += f"Color: {row['seamark:light:colour']}<br>"
    if 'seamark:light:range' in row:
        popup_text += f"Alcance: {row['seamark:light:range']} millas náuticas<br>"
    if 'seamark:light:height' in row:
        popup_text += f"Altura: {row['seamark:light:height']} m<br>"
    if 'seamark:light:period' in row:
        popup_text += f"Período: {row['seamark:light:period']} s<br>"
    if 'seamark:light:sequence' in row:
        popup_text += f"Secuencia: {row['seamark:light:sequence']}<br>"

    # Añadimos el marcador
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=folium.Popup(popup_text, max_width=300),
        icon=folium.Icon(color='orange', icon='lighthouse')
    ).add_to(m)

# Añadimos un mini mapa
minimap = plugins.MiniMap()
m.add_child(minimap)

# Añadimos control de escala
folium.plugins.MousePosition().add_to(m)
folium.plugins.MeasureControl().add_to(m)

# Añadimos diferentes capas base
folium.TileLayer('openstreetmap').add_to(m)
folium.TileLayer('cartodbdark_matter').add_to(m)

# Añadimos control de capas
folium.LayerControl().add_to(m)

# Guardamos el mapa
m.save('faros_galicia.html')

# Mostramos el mapa en el notebook
display(m)